In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from pythonlib.tools.stroketools import strokesInterpolate2, strokesFilter, smoothStrokes
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.optimize import minimize_scalar, minimize

plt.style.use('dark_background')

### On off calculation notebook
Same as the panchodate/diegodate nbs but this one is a bit cleaner and clearer for posterity sake

In [ ]:
#Data loaded here is the dat object that is returned from the process_data_singletrial function in the handtrack class
with open("/home/danhan/freiwaldDrive/dhanuska/230126_pancho_proc_data.pkl", 'rb') as f:
    dfs = pickle.load(f)

#### Five point stencil for velocity and acceleration calculation
This is an accepted method in math to calculate the velocity and acceleration for discrete data at a given point by using the surrounding points.

In [ ]:
def fps(x, fs):
    '''Five point stentil function for discrete derivative, scales to m/s auto'''
    v = [(-x[i+2] + 8*x[i+1] - 8*x[i-1] + x[i-2])/12 for i in range(len(x)) if 2<=i<len(x)-2]
    return np.array(v) * fs
def fps2(x, fs):
    '''Same as above but for second derivative scales to m/s**2 auto'''
    a = [(-x[i+2] + 16*x[i+1] - 30*x[i] + 16*x[i-1] - x[i-2])/12 for i in range(len(x)) if 2<=i<len(x)-2]
    return np.array(a) * fs**2

#### Heuristic based approach
Works decently, left it here in case it become useful later. Works by using a few observed patterns in the trajectory structure. Pretty accurate but does not handle edge cases super well.

In [ ]:
def max_dist_heuristic(x, mode):
    '''
    x, n,2 array with a value and a time
    target, target value to get close in this case a mean
    mode, onsets or offsets
    '''
    diffs = np.diff(np.diff(x[:,0]))

    if mode == 'on':
        #Return pt who has biggest difference from previous
        return x[np.argmin(diffs)+1]
    elif mode == 'off':
        #Return pt who has biggest diff from next pt
        return x[np.argmin(diffs)]
    else:
        assert False, 'give mode on or off'
    
def on_off_heuristics(stroke_vs, stroke_as, wind, t, mode, window_size = 0.0008):

    if mode == 'on':
        vz_lb = np.mean(stroke_vs) - window_size
        az_ub = np.mean(stroke_as) + window_size
        az_lb = np.mean(stroke_as) - window_size
        above_lb = wind[:,0] >= vz_lb
        a_in_thresh = (wind[:,1] >= az_lb) & (wind[:,1] <= az_ub)
        above_lb_and_no_next_below = []

        #Make sure all following vs are in thresh
        next_val_true = True
        for val in reversed(above_lb):
            if not val:
                next_val_true = False
            above_lb_and_no_next_below.append(next_val_true)
        above_lb_and_no_next_below.reverse()

        #Maker sure all following as are in thresh
        next_val_true = True
        a_in_thresh_no_next_out = []
        for val in reversed(a_in_thresh):
            if not val:
                next_val_true = False
            a_in_thresh_no_next_out.append(next_val_true)
        a_in_thresh_no_next_out.reverse()

        prev_point_more_neg = wind[:,0] > np.roll(wind[:,0],1)
        both = above_lb_and_no_next_below & prev_point_more_neg & a_in_thresh_no_next_out
        both[0] = False
        filtered = wind[both]
        if len(filtered) == 0:
            return [-100,0,t]
        else:
            pt = filtered[filtered[:,2].argsort()][0]
            return pt
    elif mode == 'off':
        vz_ub = np.mean(stroke_vs) + window_size-0.0002
        under_ub = wind[:,0] <= vz_ub
        below_ub_and_no_prev_below = []
        prev_val_true = True
        for val in under_ub:
            if not val:
                prev_val_true = False
            below_ub_and_no_prev_below.append(prev_val_true)
        next_point_more_pos = wind[:,0] < np.roll(wind[:,0],-1)
        both = below_ub_and_no_prev_below & next_point_more_pos
        both[0] = False
        filtered = wind[both]
        if len(filtered) == 0:
            return [-100,0,t]
        else:
            pt = filtered[filtered[:,2].argsort()][-1]
            return pt
    else:
        assert False, 'give mode on or off'

#### Optimize intersection method
This method seems to work the best to find the onset/offset points. it is going to take in a data structure, dat and then at each touchscreen determined onset/offset point it will fit two lines. One line follows the trajectory of the velocity into/out of the stroke, and the other fits horizontally to the velocity in the stroke. The poiint where these lines intersect is what is taken as the onset/offset. The intuition here is that we are essentially optimizing the elbow of a fitted polynomial/log function but two lines gave a more precise point rather than trying to threshold a continuous function

In [ ]:
def findFixOff(dat_trial):
    """ Function to find when on fix happens 

    Args:
        dat_trial (_type_): _description_
    """
def calcOnsetOffset(dat, search_window=0.015, do_ts_average=True, data_use = 'trans'):
    """Loops though strok data and calculates onset anbd offset points by optimizing intersection point of positive sloped and horizontal line, fitting
    the data near a stroke. 

    Args:
        dat (df or soemthing): DF holding the data needed for the computation here. Mainly we need all the fields returned from the HT.process data function
        (touch/cam positions and interpolated versions also)
        
        search_window: Numb er in seconds to tell algorithm how far to look before/after stroke to find min/max point (fitted line goes from min/max point to intercept)
        do_ts_average (bool): Average the caluclated point with the touchscreen point (helps reduce variance from less than ideal fits)
        data_use (str): Which data to use ('raw', 'trans'). Raw is raw z coord, trans is z corod after regression with transformation marix
    Returns:
        on_off_pts (dict): Dictionary containing onset and offset pts for {on_fix(offset only), strokes 1...n, off_fix(onset only)}
    """
    on_off_pts = {}

    assert len(dat) > 0, "No data here"
    if data_use == 'trans':
        strokes_cam = dat["trans_strokes_cam"]
        gaps_cam = dat["trans_gaps_cam"]
        strokes_touch = dat["strokes_touch"]
    elif data_use == 'raw':
        strokes_cam = dat["reg_strokes_cam"]
        gaps_cam = dat["reg_gaps_cam"]
        strokes_touch = dat["strokes_touch"]
    else:
        assert False, "Not sure what data you want to use"

    t_onfix_off = strokes_touch[0][-1,2]
    t_offfix_on = strokes_touch[-1][0,2]

    
    
    

